In [1]:
import numpy as np

In [2]:

def load_dataset(filename):
    file1 = open(filename, 'r')
    lines = file1.readlines()
    m = len(lines)
    arr = lines[0][:-1].split("\t")[1:]
    n = len(arr)
    D = np.zeros(n*m)
    labels = []
    for i, d in enumerate(lines):
        arr = d[:-1].split("\t")
        for j, v in enumerate(arr[1:]):
            D[i + j*m] = v
        labels.append(int(arr[0]))
    return D, labels

In [3]:
path = "D:/tesis/UCRArchive_2018/Car/Car_TRAIN.tsv"
D, labels = load_dataset(path)

In [4]:
n = int(D.shape[0] / len(labels))
m = len(labels)

In [5]:
#m: number of time series in dataset
#n: number of data in time series
def cumsum(train, m, n):
    cum1 = np.zeros(m*(n+1))
    cum2 = np.zeros(m*(n+1))
    psum = np.zeros(m)
    psum2 = np.zeros(m)
    for j in range(n):
        for i in range(m):
            psum[i] += train[i + j*m]
            psum2[i] += train[i + j*m]**2
            cum1[i + (j+1)*m] = psum[i]
            cum2[i + (j+1)*m] = psum2[i]
#     for i in range(m):
#         cum1[i + n*m] = cum1[i + (n-1)*m]
#         cum2[i + n*m] = cum2[i + (n-1)*m]
    return cum1, cum2

In [6]:
cum1, cum2 = cumsum(D, m, n)

In [7]:
cum_test = []
i = 34
for j in range(n+1):
    cum_test.append(cum2[i + j*m])
        
print(np.array(cum_test)[200])

187.84966315368402


In [8]:
bopsize = (1 << (2 * 3))

def bop(train, wd, wl, cumTrain, cumTrain2, m, n):
    bopsize = 1 << (2 * wd);
    ns = (1.0 * wl) / (1.0 * wd)
    train_bop = np.zeros((m * bopsize) + 1)
    for i in range(m):
#         counter_test = 0
        pword = -1;
        for j in range(n - wl + 1):
            sumx = cumTrain[i + (j + wl)*m] - cumTrain[i + j*m]
            sumx2 = cumTrain2[i + (j + wl)*m] - cumTrain2[i + j*m]
            meanx = (1.0 * sumx) / (wl * 1.0)
            sigmax = np.sqrt(sumx2 / wl - meanx * meanx)
            if np.isnan(sigmax):
                print(sumx2, wl, meanx, sumx2/wl, meanx*meanx, i, j)
            wordp = 0
            wc_arr = np.zeros(wd)
            for k in range(wd):
                u = round(ns * (k + 1))
                l = round(ns * k)
                sumsub = cumTrain[i + (j + u)*m] - cumTrain[i + (j+l)*m]
                avgsub = sumsub / (u - l)
                paa = (avgsub - meanx) / sigmax
                if i == 34:
                    print(j, j + wl, k, cumTrain2[i + (j + wl)*m], sumx, sumx2, paa)
#                     counter_test += 1
                if paa < 0:
                    if paa < -0.67:
                        val = 0
                    else:
                        val = 1
                else:
                    if paa < 0.67:
                        val = 2
                    else:
                        val = 3
                ws = (1 << (2 * k))*val
                wc_arr[k] = ws
                wordp += ws
            if pword != wordp:
                if i + wordp*m == 634:
                    print("634 ocurre en:", i, j, u, k, sumx, sumx2, paa)
#                 if i == 50:
#                     print("ws1: {}, ws2: {}, ws3: {}".format(wc_arr[0], wc_arr[1], wc_arr[2]),  ", j: ", j, ", i: ", i, ", wordp: ", wordp)
#                     print("sumx: ", sumx, ", sumx2: ", sumx2)
                train_bop[i + wordp*m] += 1
                pword = wordp
#         if i == 34:
#             print(counter_test)
    print(n - wl + 1)
    return train_bop

In [9]:
train_bop = bop(D, 3, int(0.25 * n), cum1, cum2, m, n)

0 144 0 129.93731880309826 43.72685920339996 129.93731880309826 1.0338034005106465
0 144 1 129.93731880309826 43.72685920339996 129.93731880309826 0.19174071173010757
0 144 2 129.93731880309826 43.72685920339996 129.93731880309826 -1.2255441122407542
1 145 0 131.26495254555226 40.70555980339996 127.77153174972202 1.0377708036991455
1 145 1 131.26495254555226 40.70555980339996 127.77153174972202 0.1858013294349411
1 145 2 131.26495254555226 40.70555980339996 127.77153174972202 -1.2235721331340867
2 146 0 132.61997519171163 37.715149503399964 125.79097852241657 1.04221580072144
2 146 1 132.61997519171163 37.715149503399964 125.79097852241657 0.17885458068181972
2 146 2 132.61997519171163 37.715149503399964 125.79097852241657 -1.2210703814032595
3 147 0 134.0013045765317 34.75486230339997 123.9861275320804 1.0480508025512432
3 147 1 134.0013045765317 34.75486230339997 123.9861275320804 0.17023784920250687
3 147 2 134.0013045765317 34.75486230339997 123.9861275320804 -1.2182886517537501
4 

159 303 1 247.63934550493738 22.584688906800004 96.46837208136617 0.20576150466303933
159 303 2 247.63934550493738 22.584688906800004 96.46837208136617 1.0443448989725959
160 304 0 248.33450887979996 24.594646286800003 95.78010595345691 -1.2539883134701315
160 304 1 248.33450887979996 24.594646286800003 95.78010595345691 0.22032865737005378
160 304 2 248.33450887979996 24.594646286800003 95.78010595345691 1.0336596561000777
161 305 0 248.97085653750452 26.567722006800004 95.03497848533459 -1.2576601892847752
161 305 1 248.97085653750452 26.567722006800004 95.03497848533459 0.2354458996624382
161 305 2 248.97085653750452 26.567722006800004 95.03497848533459 1.0222142896223372
162 306 0 249.5539141609449 28.504281006800007 94.24216130084139 -1.2610955015012393
162 306 1 249.5539141609449 28.504281006800007 94.24216130084139 0.25113935573231255
162 306 2 249.5539141609449 28.504281006800007 94.24216130084139 1.0099561457689268
163 307 0 250.13259214584158 30.43522877680001 93.451380202664

306 450 1 390.9145854060839 -28.98772373080005 141.360671245139 0.30000859020480564
306 450 2 390.9145854060839 -28.98772373080005 141.360671245139 -1.2505451186281267
307 451 0 394.52374375514614 -31.64821110080005 144.39115160930456 0.965940480002183
307 451 1 394.52374375514614 -31.64821110080005 144.39115160930456 0.28204611187400364
307 451 2 394.52374375514614 -31.64821110080005 144.39115160930456 -1.2479865918761865
308 452 0 398.13869723321574 -34.349870850800045 147.36553432017493 0.9804880159383272
308 452 1 398.13869723321574 -34.349870850800045 147.36553432017493 0.26484489057100674
308 452 2 398.13869723321574 -34.349870850800045 147.36553432017493 -1.2453329065093341
309 453 0 401.6361518972086 -37.01234987080005 150.23520101431208 0.9946943561255877
309 453 1 401.6361518972086 -37.01234987080005 150.23520101431208 0.24780170943818233
309 453 2 401.6361518972086 -37.01234987080005 150.23520101431208 -1.2424960655637698
310 454 0 404.99617690718503 -39.60122985080005 153.0

434


In [10]:
train_bop = bop(D, 3, int(0.25 * n), cum1, cum2, m, n)

0 144 0 129.93731880309826 43.72685920339996 129.93731880309826 1.0338034005106465
0 144 1 129.93731880309826 43.72685920339996 129.93731880309826 0.19174071173010757
0 144 2 129.93731880309826 43.72685920339996 129.93731880309826 -1.2255441122407542
1 145 0 131.26495254555226 40.70555980339996 127.77153174972202 1.0377708036991455
1 145 1 131.26495254555226 40.70555980339996 127.77153174972202 0.1858013294349411
1 145 2 131.26495254555226 40.70555980339996 127.77153174972202 -1.2235721331340867
2 146 0 132.61997519171163 37.715149503399964 125.79097852241657 1.04221580072144
2 146 1 132.61997519171163 37.715149503399964 125.79097852241657 0.17885458068181972
2 146 2 132.61997519171163 37.715149503399964 125.79097852241657 -1.2210703814032595
3 147 0 134.0013045765317 34.75486230339997 123.9861275320804 1.0480508025512432
3 147 1 134.0013045765317 34.75486230339997 123.9861275320804 0.17023784920250687
3 147 2 134.0013045765317 34.75486230339997 123.9861275320804 -1.2182886517537501
4 

181 325 1 255.7588504330944 59.49083821680001 78.04202708026273 0.5946610639931686
181 325 2 255.7588504330944 59.49083821680001 78.04202708026273 0.6596806416620594
182 326 0 255.90403351669426 60.77504328680001 77.37148311431488 -1.2486638232710021
182 326 1 255.90403351669426 60.77504328680001 77.37148311431488 0.6155344503472864
182 326 2 255.90403351669426 60.77504328680001 77.37148311431488 0.6331293729237157
183 327 0 256.07991099581295 62.08019887680002 76.76275775099398 -1.2426551363716918
183 327 1 256.07991099581295 62.08019887680002 76.76275775099398 0.6356499070158946
183 327 2 256.07991099581295 62.08019887680002 76.76275775099398 0.6070052293557969
184 328 0 256.28726604775545 63.40337224680001 76.2170171836274 -1.2362468421138788
184 328 1 256.28726604775545 63.40337224680001 76.2170171836274 0.6549947350656619
184 328 2 256.28726604775545 63.40337224680001 76.2170171836274 0.5812521070482171
185 329 0 256.5347172872624 64.75001654680001 75.74332847947821 -1.22946720172

357 501 1 498.51807456189715 -137.95625632080004 218.74353483311614 -0.6800087589967669
357 501 2 498.51807456189715 -137.95625632080004 218.74353483311614 -0.5399743131621763
358 502 0 498.97372021713124 -139.43164116080004 218.55858908779044 1.2109056976632893
358 502 1 498.97372021713124 -139.43164116080004 218.55858908779044 -0.7063602835872207
358 502 2 498.97372021713124 -139.43164116080004 218.55858908779044 -0.5045454140760686
359 503 0 499.38559513671726 -140.83825999080005 218.38547764774358 1.200459941319281
359 503 1 499.38559513671726 -140.83825999080005 218.38547764774358 -0.7329899231123593
359 503 2 499.38559513671726 -140.83825999080005 218.38547764774358 -0.4674700182069214
360 504 0 499.7557797557327 -142.17603638080007 218.22371317817647 1.1881909728265518
360 504 1 499.7557797557327 -142.17603638080007 218.22371317817647 -0.7594929608584816
360 504 2 499.7557797557327 -142.17603638080007 218.22371317817647 -0.4286980119680704
361 505 0 500.08634310710676 -143.44485

434


In [11]:
idx = np.where(train_bop != 0)[0]
print(len(idx), np.sum(train_bop))
print(idx[0:100])

1100 1722.0
[420 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437
 438 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455
 456 457 458 459 460 461 462 463 464 465 466 467 468 469 470 471 472 473
 474 475 476 477 478 479 600 601 602 603 604 605 606 607 608 609 610 611
 612 613 614 615 616 617 618 619 620 621 622 623 624 625 626 627 628 629
 630 631 632 633 634 635 636 637 638 639]


In [12]:
def adjustlabelset(train_label, m):
    train_label_index = np.zeros(m, dtype=int)
    tlabel = np.sort(np.unique(train_label))
    class_count = np.zeros(tlabel.shape[0])
    for i in range(m):
        position = int(np.where(tlabel == train_label[i])[0][0])
        train_label_index[i] = position
        class_count[position] += 1
    return class_count, train_label_index, tlabel

In [13]:
class_count, train_label_index, tlabel = adjustlabelset(labels, m)
c = len(tlabel)

In [14]:
class_count

array([16., 16., 11., 17.])

In [15]:
def anova(train_bop, train_label_index, class_count, m, n, c):
    bop_f_a = np.zeros(bopsize)
    bop_f_a_i = 0
    for j in range(n):
        sumall = 0.0
        sumc = np.zeros(c)
        for i1 in range(m):
            k = int(train_label_index[i1])
            sumall += train_bop[i1 + j*m]
            sumc[k] += train_bop[i1 + j*m]
        
        avgall = sumall / m;
        ssa = 0.0
        avgc = np.zeros(c)
        for k in range(c):
            avgc[k] = sumc[k] / class_count[k]
            ssa += class_count[k] * (avgc[k] - avgall)*(avgc[k] - avgall)
        
        ssw = 0
        for i2 in range(m):
            k = train_label_index[i2]
            ssw += (train_bop[i2 + j*m] - avgc[k])*(train_bop[i2 + j*m] - avgc[k])
        
        msa = 1.0*ssa / (1.0*c - 1)
        msw = 1.0*ssw / (1.0*m - 1.0*c)
        if not (msa == 0 and msw == 0):
            bop_f_a[bop_f_a_i] = msa / msw
        bop_f_a_i += 1
    return bop_f_a

In [16]:
bop_f_a = anova(train_bop, train_label_index, class_count, m, bopsize, c)

In [17]:
anova_rank_sort = np.argsort(bop_f_a)[::-1]
idx  = 0
while bop_f_a[anova_rank_sort[idx]] > 0:
    idx += 1
rank_idx = anova_rank_sort[:idx]
idx, bop_f_a[rank_idx]

(24, array([12.39560413, 11.74678181,  8.81088435,  8.29920874,  7.90106927,
         5.71666667,  4.08172204,  3.6743979 ,  3.64809741,  3.38765432,
         3.38765432,  3.37464613,  3.27509025,  3.27274099,  2.96963454,
         2.40356316,  2.23889505,  1.95555556,  1.73521127,  1.43287349,
         1.15924905,  0.74446086,  0.47638889,  0.29125296]))

In [18]:
def sort_trim_arr(train_bop, sort_index, m, n):
    train_bop_sort = []
    for j in range(n):
        k = sort_index[j]
        for i in range(m):
            train_bop_sort.append(train_bop[i + k*m])
    return np.array(train_bop_sort)

In [19]:
train_bop_sort = sort_trim_arr(train_bop, anova_rank_sort, m, idx)

In [20]:
train_bop_sort

array([1., 1., 1., ..., 2., 2., 2.])

In [21]:
# def cross_vl_centroid(train_bop_sort, train_label_index, class_count, m, n, c):
#     for k in range(n):
#         x = np.zeros((c, m))
        
#         for i in range(m):
#             p = train_label_index[i]

In [22]:
def crossVL(train_bop_sort, train_label_index, class_count, m, n, c):
    x = np.zeros((c, m))
    y = np.zeros((m, c))
    crossL = []
    maxcount = -1
    for k in range(n):
        count = 0
        for i in range(m):
            p = train_label_index[i]
            x[p][k] += train_bop_sort[i + k*m]
        for i in range(c):
            x[i][k] = x[i][k] / class_count[i]
            crossL.append(x[i][k])
        for i in range(m):
            rmin = np.inf
            label = 0.0
            p = train_label_index[i]
            countc = class_count[p]
            for j in range(c):
                r = y[i][j]
                pm = train_bop_sort[i + k*m]
                d = pm -x[j][k]
                if j == p:
                    d += pm / countc
                r += d*d
                y[i][j] = r
                if (r < rmin):
                    rmin = r
                    label = j
            if label == train_label_index[i]:
                count += 1
        if count >= maxcount:
            maxcount = count
            maxk = k
    
    best_idx = maxk + 1
    best_score = maxcount / m
    
    return crossL, best_idx, best_score, maxcount

In [23]:
crossL, best_idx, best_score, maxcount = crossVL(train_bop_sort, train_label_index, class_count, m, idx, c)